<a href="https://colab.research.google.com/github/HarryHy/Neuro140/blob/main/%E2%80%9CNeuro_140_Style__transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

import cv2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
imsize = 512 if torch.cuda.is_available() else 256  # use small size if no gpu

loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # fake batch dimension required to fit network's input dimensions
    image = Image.fromarray(image)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [ ]:
root_file = "/content/gdrive/My Drive/noreward-rl-master/"
style_img = image_loader("/content/gdrive/My Drive/c2.jpeg")
style_img_grass = image_loader("/content/gdrive/My Drive/g1.jpeg")
style_img_elephant = image_loader("/content/gdrive/My Drive/e1.jpeg")
style_img_bottle = image_loader("/content/gdrive/My Drive/b1.jpeg")
style_img_paint = image_loader("/content/gdrive/My Drive/m1.jpeg")
content_img = image_loader("/content/gdrive/My Drive/g1.jpeg")

In [ ]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated
    return image

plt.figure()
imshow(style_img, title='Style Image')

plt.figure()
imshow(content_img, title='Content Image')

In [ ]:
class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [ ]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img])
    return optimizer

In [ ]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

In [ ]:
class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [ ]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [ ]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

In [ ]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1):
    """Run the style transfer."""
    #print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)

    # We want to optimize the input and not the model parameters so we
    # update all the requires_grad fields accordingly
    input_img.requires_grad_(True)
    model.requires_grad_(False)

    optimizer = get_input_optimizer(input_img)

    #print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            with torch.no_grad():
                input_img.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            
            if run[0] % 100 == 0:
                #print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                #print()
            
            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    with torch.no_grad():
        input_img.clamp_(0, 1)

    return input_img

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

In [ ]:
from torchvision import models
dir(models)
resnet = models.resnext101_32x8d(pretrained=True)

In [ ]:
dir(models)

In [ ]:
import numpy as np

# Clock Example

In [ ]:
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/Clock/"

for f in os.listdir(path):
    count = 0
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    for num in [0, 20, 50, 100, 300, 500]:
        print("num is ", num)
        if count > 20:
            break
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_grass, input_img, num_steps= num)

          plt.figure()
          imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)

        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
        img_cat_preprocessed = preprocess(p_out)
        batch_img_cat_tensor = torch.unsqueeze(img_cat_preprocessed, 0)

        resnet.eval()
        out = resnet(batch_img_cat_tensor)
        _, index = torch.max(out, 1)
        print(index)
    count += 1

# Elephant Example

In [ ]:
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"

for f in os.listdir(path):
    count = 0
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    for num in [0, 20, 100, 500, 750]:
        print("num is ", num)
        if count > 30:
            break
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img, input_img, num_steps= num)

          plt.figure()
          imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)

        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
        img_cat_preprocessed = preprocess(p_out)
        batch_img_cat_tensor = torch.unsqueeze(img_cat_preprocessed, 0)

        resnet.eval()
        out = resnet(batch_img_cat_tensor)
        #_, index = torch.max(out, 1)
        
        accuracy, index = torch.topk(out, 5, dim = 1)
        print(accuracy)
        print(index)
    count += 1

In [ ]:
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"
count = 0

for f in os.listdir(path):
    print("nun of style is ", count)
    if count > 25:
            break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    for num in [0, 20, 100, 500, 750]:
        print("num is ", num)
        
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_grass, input_img, num_steps= num)

          plt.figure()
          imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)

        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
        img_cat_preprocessed = preprocess(p_out)
        batch_img_cat_tensor = torch.unsqueeze(img_cat_preprocessed, 0)

        resnet.eval()
        out = resnet(batch_img_cat_tensor)
        #_, index = torch.max(out, 1)
        
        accuracy, index = torch.topk(out, 5, dim = 1)
        print(accuracy)
        print(index)
    count += 1

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"
count = 0

for f in os.listdir(path):
    if count > 25:
            break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        if count > 25:
            break
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_elephant, input_img, num_steps= num)

          plt.figure()
          imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)

        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
        

        img_cat_preprocessed = preprocess(p_out)
        batch_img_cat_tensor = torch.unsqueeze(img_cat_preprocessed, 0)

        resnet.eval()
        out = resnet(batch_img_cat_tensor)
        #_, index = torch.max(out, 1)
        
        accuracy, index = torch.topk(out, 5, dim = 1)
        print(accuracy)
        print(index)
    count += 1

In [ ]:
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"
count = 0

for f in os.listdir(path):
    if count > 25:
            break
    print("count is ", count)
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        if count > 25:
            break
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_leo, input_img, num_steps= num)

          plt.figure()
          imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)

        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
        img_cat_preprocessed = preprocess(p_out)
        batch_img_cat_tensor = torch.unsqueeze(img_cat_preprocessed, 0)

        resnet.eval()
        out = resnet(batch_img_cat_tensor)
        #_, index = torch.max(out, 1)
        
        accuracy, index = torch.topk(out, 5, dim = 1)
        print(accuracy)
        print(index)
    count += 1

In [ ]:
style_img_leo = image_loader("/content/gdrive/My Drive/l1.jpeg")

# Save the Image

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"
count = 0

for f in os.listdir(path):
    if count > 50:
            break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        if count > 25:
            break
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_elephant, input_img, num_steps= num)

          plt.figure()
          imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)

        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
        
        p_out.save("/content/gdrive/My Drive/Style/elephant/" + "elephant_" + str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    count += 1

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"
count = 0

for f in os.listdir(path):
    if count > 25:
            break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        if count > 25:
            break
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img, input_img, num_steps= num)

          plt.figure()
          imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)

        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
        
        p_out.save("/content/gdrive/My Drive/Style/cat/" + str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    count += 1

#Airplane texture generator


In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/Airplane/"
count = -1

for f in os.listdir(path):
    count += 1
    if count > 90:
      break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    if count <= 50:
      continue
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/train/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/train/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/Airplane/"
count = -1

for f in os.listdir(path):
    count += 1
    if count > 105:
      break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    if count <= 90:
      continue
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/val/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/val/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

# Orange Generator
Training data

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/Orange/"
count = -1

style_img_orang = style_img
for f in os.listdir(path):
    count += 1
    if count > 100:
      break
    if count <= 90:
      continue
    if count > 90:
      style_img_orang = style_img_paint
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_orang, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/train/orange/" + "orange_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/train/orange/" + "orange_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

# Orange 
test data

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/Orange/"
count = -1
style_img_orang = style_img
for f in os.listdir(path):
    count += 1
    if count > 120:
      break
    if count <= 100:
      print(count)
      continue
    if count > 110:
      style_img_orang = style_img_paint
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_orang, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/val/orange/" + "orange_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/val/orange/" + "orange_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

# Guitar Generator

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/Guitar/"
count = -1
style_img_guitar = style_img
for f in os.listdir(path):
    count += 1
    if count > 100:
      break
    
    if count <= 95:
      continue

    if count > 75:
      style_img_guitar = style_img_elephant
    if count > 90:
      style_img_guitar = style_img_paint
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_guitar, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/train/Guitar/" + "guitar_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/train/Guitar/" + "guitar_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/Guitar/"
count = -1
style_img_guitar = style_img
for f in os.listdir(path):
    count += 1
    
    if count > 125:
      break
    if count <= 100:
      print(count)
      continue
    if count >112:
      style_img_guitar = style_img_paint
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img_guitar, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/val/Guitar/" + "guitar_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/val/Guitar/" + "guitar_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

# Elephant generator

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"
count = -1

for f in os.listdir(path):
    count += 1
    if count > 90:
      break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    if count <= 50:
      continue
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/train/elephant/" + "elephant_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/train/elephant/" + "elephant_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

path = "/content/gdrive/My Drive/Image20/elephant/"
count = -1

for f in os.listdir(path):
    count += 1
    if count > 105:
      break
    content_img = image_loader(str(path+f))
    input_img = content_img.clone()
    imshow(content_img)
    if count <= 90:
      continue
    for num in [0, 20, 100, 500]:
        print("num is ", num)
        print(str(path+f))
        if f == ".DS_Store":
            continue
        
        if num !=0 :
          output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                                  content_img, style_img, input_img, num_steps= num)

          plt.figure()
          img = imshow(output, title='Output Image')
          plt.ioff()
          plt.show()

          img.save("/content/gdrive/My Drive/p/val/elephant/" + "elephant_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          '''
          n_out = output.cpu().detach().numpy()
          #print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          #print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          '''
        else:

          n_out = content_img.cpu().detach().numpy()
          print(n_out.shape)
          n_out = n_out.reshape((imsize, imsize, 3))
          print(n_out.shape)

          n_out = n_out * 255
          n_out = n_out.astype(np.uint8)

          p_out = Image.fromarray(n_out)
          p_out.save("/content/gdrive/My Drive/p/val/elephant/" + "elephant_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    

# DCT Transformation

In [ ]:
import numpy as np

def rgb2yuv(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    Y=0.299*r+0.587*g+0.114*b
    U=-0.147*r-0.289*g+0.436*b
    V=0.615*r-0.515*g-0.100*b

    return Y, U, V

In [ ]:
#@title
from PIL import Image
import glob
import os, os.path

from PIL import Image as PImage

path = "/content/gdrive/My Drive/Image20/Airplane/"
count = -1

for f in os.listdir(path):
    count += 1
    if count > 500:
      break
    if count < 300:
      continue
    


    #input_img = content_img.clone()

    #print(type(input_img))
    image = cv2.imread(str(path+f))
    #plt.imshow(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #plt.imshow(image)
    new_img = cv2.resize(image, (512, 512))
    plt.imshow(new_img)
    plt.pause(0.001)
    Y, U, V  = cv2.split(new_img)
    Y = np.float32(Y)
    U = np.float32(U)
    V = np.float32(V)
    #Y, U, V = rgb2yuv(new_img)
    dcty = cv2.dct(Y)
    dctu = cv2.dct(U)
    dctv = cv2.dct(V)
    for i in range(512):
      for j in range(512):
          if i> 32 or j > 32:
              dcty[i][j] = 0
              dctu[i][j] = 0
              dctv[i][j] = 0
    dct2y = cv2.idct(dcty)
    dct2u = cv2.idct(dctu)
    dct2v = cv2.idct(dctv)
    
    rgb_uint8 = (np.dstack((dct2y, dct2u, dct2v))).astype(np.uint8) 

    plt.imshow(rgb_uint8)
    plt.pause(0.001)
    
    plt.imshow(dct2y)
    
    plt.pause(0.001)
    '''
    #Image.open(path).convert('RGB')
    #plt.imshow(dct2y3)
    stacked_img = np.stack((dct2y,)*3, axis=-1)

    im = Image.fromarray((stacked_img  * 255).astype(np.uint8))
    #imrgb = im.convert('RGB')
    plt.imshow(im)
    '''
    #im.save("/content/gdrive/My Drive/dp/train/elephant/" + "elephant_" +str(count) + "_" + "iter_"  + ".jpeg")
    #im.save("/content/gdrive/My Drive/dp/train/airplane/" + "airplane_" +str(count) + "_" + "iter_"  + ".jpeg")
          #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        #p_out.save("/content/gdrive/My Drive/Style/airplane/" + "airplane_" +str(count) + "_" + "iter_" + str(num) + ".jpeg")
        
    